In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


df_full = pd.read_csv('../data/The_Dataset_v1-1.csv', parse_dates = ["Date"])
df = df_full.copy()
# remove NAs
df = df.dropna().copy()

df.describe()

In [ ]:
# Clip some values of cloud cover mean which are >100 to 100
df['cloud_cover_mean'] = df['cloud_cover_mean'].clip(upper=100)
# Remove some rows where cloud cover min is -1
df = df[df['cloud_cover_min'] != -1]

In [ ]:
# Variable summaries look good, now histograms
df.hist(bins=25, figsize=(15, 10))
plt.tight_layout()
plt.show()

In [ ]:
# Histograms look good.
# The weather code variable needs to be mapped onto categories (currently has numbers representing weather codes)
# Start by looking at the frequencies of each weather code
print(df['weather_code'].value_counts().sort_index())

# Boxplots of efficiency by weather code
plt.figure(figsize=(12, 6))
sns.boxplot(data=df, x='weather_code', y='Efficiency (kWh/kW)')
plt.xlabel('Weather Code')
plt.ylabel('Efficiency (kWh/kW)')
plt.xticks(rotation=90)
plt.show()

In [ ]:
# The following mapping of weather codes looks appropriate and leaves no sparse categories
weather_map = {
    0: 'clear',
    1: 'partly_cloudy', 2: 'partly_cloudy',
    3: 'overcast',
    45: 'fog', 48: 'fog',
    51: 'drizzle', 53: 'drizzle', 55: 'drizzle',
    56: 'drizzle', 57: 'drizzle',
    61: 'rain', 63: 'rain', 65: 'rain',
    66: 'rain', 67: 'rain',
    71: 'snow', 73: 'snow', 75: 'snow',
    77: 'snow',
    80: 'rain_showers', 81: 'rain_showers', 82: 'rain_showers',
    85: 'snow', 86: 'snow',
    95: 'thunderstorm',
    96: 'thunderstorm', 99: 'thunderstorm'
}
df['weather_category'] = df['weather_code'].map(weather_map)
df = pd.get_dummies(df, columns=['weather_category'], drop_first=True)
df = df.drop(columns=['weather_code'])

In [ ]:
# calculate correlations, excluding the new dummy variables
cols_to_exclude = [col for col in df.columns if col.startswith('weather_category_')]
corr_matrix = df.drop(columns = cols_to_exclude).corr()

# plot heatmap of the correlations of all pairs of variables
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title("Correlation Heatmap")
plt.show()

In [ ]:
# pair plot of the variables with the strongest linear correlations with efficiency (plotting everything is too much)
sns.pairplot(df[["Efficiency (kWh/kW)", "sunshine_duration", "daylight_duration",
                 "relative_humidity_2m_min", "shortwave_radiation_sum", "cloud_cover_mean"]],
            plot_kws={"s": 5})

In [ ]:
# Noticing some lines in the plots with efficiency, let's see where they are:
print(df["Efficiency (kWh/kW)"].value_counts())

In [ ]:
# 2.278 is a very frequent value, is it associated with particular dates or a system?
print(df_full[df_full["Efficiency (kWh/kW)"] == 2.278]["Date"].value_counts())
print(df_full[df_full["Efficiency (kWh/kW)"] == 2.278]["System ID"].value_counts())

In [ ]:
# System 3099 has 180 of the 191 instances of 2.278.
# Let's see what is going on for 3099:
value_counts = df_full[df_full["System ID"] == 3099]["Efficiency (kWh/kW)"].value_counts()

sns.barplot(x=value_counts.index, y=value_counts.values)
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Clearly, there should be more unique values for efficiency over thousands of days.
# Let's remove system 3099 (in a new dataframe).
df_edit = df.copy()
df_edit = df_edit[df_edit["System ID"] != 3099]

In [ ]:
# Let's see if there are other systems with very few unique values of efficiency:
all_sids = df_edit["System ID"].drop_duplicates()
sid_unique_counts = []

for sid in all_sids:
    unique_count = df_edit[df_edit["System ID"] == sid]["Efficiency (kWh/kW)"].nunique()
    sid_unique_counts.append((unique_count, sid))

# Print the sorted results
sid_unique_counts.sort()
for unique_count, sid in sid_unique_counts:
    print(f"{unique_count} unique values for System ID {sid}")

In [ ]:
# Plot bar charts for systems with very few unique values of efficiency.
for sid in (8224, 4113, 32351, 6090, 46979, 3641):
    print(f"{len(df_edit[df_edit["System ID"] == sid]["Efficiency (kWh/kW)"].unique())} unique values for System ID {sid}")
    value_counts = df_edit[df_edit["System ID"] == sid]["Efficiency (kWh/kW)"].value_counts()

    sns.barplot(x=value_counts.index, y=value_counts.values)
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
# The first two sids have 0 efficiency every day. The last four sids have a small set of unique efficiency values.
sids_to_delete = (8224, 4113, 32351, 46979, 3641, 6090)
for sid in sids_to_delete:
    df_edit = df_edit[df_edit["System ID"] != sid]

In [ ]:
# Let's take another look at the frequency of the efficiency values
print(df_edit["Efficiency (kWh/kW)"].value_counts().head(50))

In [ ]:
# Check these values of efficiency by system ID
for value in (0, 0.3, 0.475, 0.2, 0.6, 0.4, 0.42, 2.513, 1.525, 0.15, 0.21, 1.425, 0.145, 2.079):
    print(f"Efficiency - {value}, top 5 contributors")
    print(df_edit[df_edit["Efficiency (kWh/kW)"] == value]["System ID"].value_counts().head(5))

In [ ]:
# 37543 has a period where there is 0 efficiency and "not sure" recorded in the condition column.
# Looking at the systems 51963 and 5116, the two systems with the most 0s after 37543, we see that all of them have "not sure"
# in the condition column in pvoutput.org, and generally occur on consecutive days, suggesting repairs/maintenance/deactivation.
# Let's drop all rows where efficiency is 0:
df_edit = df_edit[df_edit["Efficiency (kWh/kW)"] != 0]

In [ ]:
# Plot some more systems that look suspect
for sid in (86644, 6125, 3764, 51963, 5116, 3036, 66986):
    print(f"{len(df_edit[df_edit["System ID"] == sid]["Efficiency (kWh/kW)"].unique())} unique values for System ID {sid}")
    value_counts = df_edit[df_edit["System ID"] == sid]["Efficiency (kWh/kW)"].value_counts()

    sns.barplot(x=value_counts.index, y=value_counts.values)
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
# None of these systems look erroneous according to these plots, they just have low precision values.
# No issues when inspecting the systems on pvoutput.org either.
# Finally, check a few more systems with relatively few unique values of efficiency.
for sid in (14745, 42343, 106682, 22778):
    print(f"{len(df_edit[df_edit["System ID"] == sid]["Efficiency (kWh/kW)"].unique())} unique values for System ID {sid}")
    value_counts = df_edit[df_edit["System ID"] == sid]["Efficiency (kWh/kW)"].value_counts()

    sns.barplot(x=value_counts.index, y=value_counts.values)
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
# No issues after looking at them all in pvoutout.org.
# Now that the data are clean, let's check the pair plot again to make sure the lines of equal efficiency are gone:
sns.pairplot(df[["Efficiency (kWh/kW)", "sunshine_duration", "daylight_duration",
                 "relative_humidity_2m_min", "shortwave_radiation_sum", "cloud_cover_mean"]],
            plot_kws={"s": 5})

In [ ]:
# Finally, add functions to "data_cleaning.py" in "build_dataset" package to automatically do all the data cleaning in this notebook.